<a href="https://colab.research.google.com/github/karanjkar99/Finance/blob/master/Portfolio_Opt_(by_3_methods)_predetermined_sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [7]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=3a5c32d289249cbdaa18d6cd1649e836f52a5f9b2eda6b31fcc88c5e08ca0c9f
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [9]:
yf.pdr_override()

In [34]:
syms = ['TIGLOB.BO','UFO.NS','ALEMBICLTD.NS','GANESHBE.BO','ESTER.NS','INTENTECH.BO','UNIDT.BO','SEAMECLTD.BO','CHAMANSEQ.BO']

In [35]:
print (syms)

['TIGLOB.BO', 'UFO.NS', 'ALEMBICLTD.NS', 'GANESHBE.BO', 'ESTER.NS', 'INTENTECH.BO', 'UNIDT.BO', 'SEAMECLTD.BO', 'CHAMANSEQ.BO']


In [36]:
start = "2016-04-01"
end = str(int(start[:4]) + 3) + start[4:]
stock_data = pdr.get_data_yahoo(syms[1:], start = start, end = end)['Adj Close']
stock_data.head()

[*********************100%***********************]  8 of 8 completed


,ALEMBICLTD.NS,CHAMANSEQ.BO,ESTER.NS,GANESHBE.BO,INTENTECH.BO,SEAMECLTD.BO,UFO.NS,UNIDT.BO
Date,,,,,,,,
2016-04-01,34.972027,53.022850,56.922913,20.600000,39.733334,69.800003,328.910583,23.107033
2016-04-04,35.751347,52.038460,56.824432,22.250000,39.733334,71.400002,359.472015,23.083946
2016-04-05,35.118145,52.038460,54.116161,21.600000,39.335999,71.099998,339.622070,23.083946
2016-04-06,34.923317,52.396473,55.494915,21.450001,38.541332,72.500000,346.700592,23.083946
2016-04-07,34.728489,51.635742,54.116161,21.100000,38.740002,72.199997,353.591919,23.083946


In [37]:
returns = stock_data.pct_change()
returns.head()

,ALEMBICLTD.NS,CHAMANSEQ.BO,ESTER.NS,GANESHBE.BO,INTENTECH.BO,SEAMECLTD.BO,UFO.NS,UNIDT.BO
Date,,,,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-04,0.022284,-0.018565,-0.001730,0.080097,0.000000,0.022923,0.092917,-0.000999
2016-04-05,-0.017711,0.000000,-0.047660,-0.029213,-0.010000,-0.004202,-0.055220,0.000000
2016-04-06,-0.005548,0.006880,0.025478,-0.006944,-0.020202,0.019691,0.020842,0.000000
2016-04-07,-0.005579,-0.014519,-0.024845,-0.016317,0.005155,-0.004138,0.019877,0.000000


In [38]:
mean_daily_returns = np.array(returns.mean()).reshape(-1, 1)
cov = returns.cov()

In [39]:
mean_daily_returns

array([[ 0.00053616],
       [ 0.00113528],
       [-0.00012422],
       [ 0.00173295],
       [ 0.00072609],
       [ 0.00285538],
       [-0.00047637],
       [ 0.00333254]])

In [40]:
cov

,ALEMBICLTD.NS,CHAMANSEQ.BO,ESTER.NS,GANESHBE.BO,INTENTECH.BO,SEAMECLTD.BO,UFO.NS,UNIDT.BO
ALEMBICLTD.NS,0.000655,0.000168,0.000223,0.000181,0.000149,0.000109,0.000148,0.000161
CHAMANSEQ.BO,0.000168,0.001025,0.000181,0.000158,0.000114,0.000140,0.000147,0.000144
ESTER.NS,0.000223,0.000181,0.001163,0.000206,0.000240,0.000172,0.000152,0.000059
GANESHBE.BO,0.000181,0.000158,0.000206,0.001218,0.000172,0.000166,0.000174,0.000073
INTENTECH.BO,0.000149,0.000114,0.000240,0.000172,0.001642,0.000126,0.000091,0.000009
SEAMECLTD.BO,0.000109,0.000140,0.000172,0.000166,0.000126,0.000811,0.000111,0.000083
UFO.NS,0.000148,0.000147,0.000152,0.000174,0.000091,0.000111,0.000450,0.000099
UNIDT.BO,0.000161,0.000144,0.000059,0.000073,0.000009,0.000083,0.000099,0.002869


### Max Sharpe Ratio

In [42]:
srs = []
portfolio_stds = []
rand_wts = []
portfolio_returns = []
risk_free_rate = 0
for i in range(0, 20000):
    random_weights = np.random.dirichlet(np.ones(8), size = 1).T
    rand_wts.append(random_weights)
    # portolfio return
    portfolio_return = np.sum(mean_daily_returns * random_weights)*252
    portfolio_returns.append(portfolio_return)
    # portfolio volatility
    portfolio_std = np.sqrt(np.dot(random_weights.T, np.dot(cov, random_weights))) * np.sqrt(252)
    portfolio_stds.append(portfolio_std)
    # sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    srs.append(sharpe_ratio)
max_index = srs.index(max(srs))
best_wts = rand_wts[max_index]
max_sr = srs[max_index]
portfolio_sd = portfolio_stds[max_index]
max_return = portfolio_returns[max_index]
print(f"Max Sharpe Ratio: {max_sr}")
print(f"Best Weights: {best_wts.T}")
print(f"Portfolio Volatility: {portfolio_sd*100}%, Sum of weights: {np.sum(best_wts)}")
print(f"Max Expected Annual Return: {max_return*100}%")

Max Sharpe Ratio: [[1.82312509]]
Best Weights: [[2.35294277e-02 3.74370698e-02 5.19288777e-02 3.52905249e-02
  3.31602093e-02 6.39301845e-01 5.52308956e-04 1.78799737e-01]]
Portfolio Volatility: [[35.31548193]]%, Sum of weights: 0.9999999999999998
Max Expected Annual Return: 64.38454112600382%


### Minimum Variance

In [43]:
returns.std()

ALEMBICLTD.NS    0.025602
CHAMANSEQ.BO     0.032008
ESTER.NS         0.034109
GANESHBE.BO      0.034903
INTENTECH.BO     0.040517
SEAMECLTD.BO     0.028478
UFO.NS           0.021202
UNIDT.BO         0.053567
dtype: float64

In [44]:
stds = np.array(returns.std()).reshape(-1, 1)
stds.shape

(8, 1)

In [45]:
product_std = np.dot(stds, stds.T)
product_std

array([[0.00065544, 0.00081947, 0.00087326, 0.00089358, 0.00103731,
        0.00072909, 0.0005428 , 0.00137139],
       [0.00081947, 0.00102454, 0.00109178, 0.0011172 , 0.0012969 ,
        0.00091155, 0.00067863, 0.00171458],
       [0.00087326, 0.00109178, 0.00116345, 0.00119053, 0.00138202,
        0.00097138, 0.00072318, 0.00182712],
       [0.00089358, 0.0011172 , 0.00119053, 0.00121825, 0.0014142 ,
        0.00099399, 0.00074001, 0.00186966],
       [0.00103731, 0.0012969 , 0.00138202, 0.0014142 , 0.00164166,
        0.00115387, 0.00085904, 0.00217038],
       [0.00072909, 0.00091155, 0.00097138, 0.00099399, 0.00115387,
        0.00081102, 0.00060379, 0.00152549],
       [0.0005428 , 0.00067863, 0.00072318, 0.00074001, 0.00085904,
        0.00060379, 0.00044951, 0.00113571],
       [0.00137139, 0.00171458, 0.00182712, 0.00186966, 0.00217038,
        0.00152549, 0.00113571, 0.00286939]])

In [46]:
cov_mat = np.array(cov)
cov_mat

array([[6.55443748e-04, 1.68114450e-04, 2.23144269e-04, 1.80577633e-04,
        1.48705520e-04, 1.08767645e-04, 1.47991739e-04, 1.61411325e-04],
       [1.68114450e-04, 1.02453575e-03, 1.80698754e-04, 1.58085988e-04,
        1.13561358e-04, 1.40397784e-04, 1.46516420e-04, 1.43837614e-04],
       [2.23144269e-04, 1.80698754e-04, 1.16344816e-03, 2.05843857e-04,
        2.39783144e-04, 1.71502920e-04, 1.52453613e-04, 5.92440908e-05],
       [1.80577633e-04, 1.58085988e-04, 2.05843857e-04, 1.21824909e-03,
        1.71575082e-04, 1.65703777e-04, 1.74037860e-04, 7.28554255e-05],
       [1.48705520e-04, 1.13561358e-04, 2.39783144e-04, 1.71575082e-04,
        1.64165860e-03, 1.25967677e-04, 9.13027885e-05, 8.78727600e-06],
       [1.08767645e-04, 1.40397784e-04, 1.71502920e-04, 1.65703777e-04,
        1.25967677e-04, 8.11019940e-04, 1.11498918e-04, 8.29499710e-05],
       [1.47991739e-04, 1.46516420e-04, 1.52453613e-04, 1.74037860e-04,
        9.13027885e-05, 1.11498918e-04, 4.49512888e-04, 9.

#### Correlation Matrix

In [47]:
corr = cov_mat / product_std
corr

array([[1.        , 0.20515106, 0.25553154, 0.20208226, 0.14335669,
        0.14918194, 0.27264572, 0.11769872],
       [0.20515106, 1.        , 0.16550765, 0.14150174, 0.087564  ,
        0.15402135, 0.21589951, 0.0838908 ],
       [0.25553154, 0.16550765, 1.        , 0.17290054, 0.17350169,
        0.1765559 , 0.21081093, 0.03242477],
       [0.20208226, 0.14150174, 0.17290054, 1.        , 0.12132345,
        0.16670499, 0.23518229, 0.03896721],
       [0.14335669, 0.087564  , 0.17350169, 0.12132345, 1.        ,
        0.10916965, 0.10628489, 0.00404873],
       [0.14918194, 0.15402135, 0.1765559 , 0.16670499, 0.10916965,
        1.        , 0.18466467, 0.05437583],
       [0.27264572, 0.21589951, 0.21081093, 0.23518229, 0.10628489,
        0.18466467, 1.        , 0.08760743],
       [0.11769872, 0.0838908 , 0.03242477, 0.03896721, 0.00404873,
        0.05437583, 0.08760743, 1.        ]])

In [49]:
port_vars = []
rand_wts = []
for i in range(0, 10000):
    random_weights = np.random.dirichlet(np.ones(8), size = 1).T
    rand_wts.append(random_weights)
    random_weighted_sd = stds * random_weights
    portfolio_var = np.sqrt(np.sum(np.dot(random_weighted_sd.T, np.dot(corr, random_weighted_sd))))*100
    port_vars.append(portfolio_var)
min_index = port_vars.index(min(port_vars))
best_wts = rand_wts[min_index]
min_var = port_vars[min_index]
portfolio_return = np.sum(mean_daily_returns * best_wts)*252
print(f"Best Weights: {best_wts}, Minimum Portfolio Variance: {min_var}, Sum of weights: {np.sum(best_wts)}")
print(f"Return: {portfolio_return*100}%")

Best Weights: [[0.13462501]
 [0.10217224]
 [0.08507919]
 [0.0646705 ]
 [0.09611   ]
 [0.13644466]
 [0.36521322]
 [0.01568518]], Minimum Portfolio Variance: 1.5534484681654375, Sum of weights: 1.0
Return: 15.809375195341863%


### Monthly, Quarterly, Half-Yearly, Yearly

In [50]:
e = stock_data
monthly = 0
quarterly = 0
half_yearly = 0
yearly = 0
# monthly
start = "2016-04-01"
for i in range(36):
    end = start[:5]
    month = int(start[5:7]) + 1
    if month <= 9:
        end += '0' + str(month) + start[7:]
    elif month > 12:
        end = str(int(start[:4]) + 1) + '-01-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    monthly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# quarterly
start = "2016-04-01"
for i in range(12):
    end = start[:5]
    month = int(start[5:7]) + 3
    if month <= 9:
        end += '0' + str(month) + start[7:]
    elif month == 13:
        end = str(int(start[:4]) + 1) + '-01-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    quarterly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# half-yearly
start = "2016-04-01"
for i in range(6):
    end = start[:5]
    month = int(start[5:7]) + 6
    if month == 16:
        end = str(int(start[:4]) + 1) + '-04-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    half_yearly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# yearly
start = "2016-04-01"
for i in range(3):
    end = str(int(start[:4]) + 1) + start[4:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    yearly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end

avgs = [list(monthly*100/36), list(quarterly*100/12), list(half_yearly*100/6), list(yearly*100/3)]
print(f"Monthly Avg Returns: {avgs[0]}")
print(f"Quarterly Avg Return: {avgs[1]}")
print(f"Half Yearly Avg Return: {avgs[2]}")
print(f"Yearly Avg Return: {avgs[3]}")

Monthly Avg Returns: [0.803162962986533, 1.7694473193963947, -0.2596859755250684, 3.122047517038763, 0.7510716460082639, 5.560015315588121, -0.8853314180080418, 5.494736412621707]
Quarterly Avg Return: [2.428652595805809, 6.574126342742551, -1.4481197947844249, 10.163704774337253, 5.590376141929499, 20.449878048433213, -2.898010603842211, 16.116678567618795]
Half Yearly Avg Return: [3.208252537101867, 13.62137260457368, -1.828869512199238, 30.789411554110988, 12.788974024813148, 36.179830207220114, -8.200415002992969, 39.77603101749069]
Yearly Avg Return: [6.816663818436941, 30.747467363367864, -8.510082974371683, 53.84021065479808, 61.58861990527353, 81.21287108091326, -15.416514212690146, 129.42213231444126]


In [51]:
for avg in avgs:
    random_weights = np.random.dirichlet(np.ones(8), size = 1).T
    total_return = avg * random_weights.T
    print(avg)
    print(random_weights.T)
    print(total_return)
    print(np.sum(total_return))

[0.803162962986533, 1.7694473193963947, -0.2596859755250684, 3.122047517038763, 0.7510716460082639, 5.560015315588121, -0.8853314180080418, 5.494736412621707]
[[0.03740026 0.28674708 0.18272598 0.1268413  0.05886588 0.12671493
  0.0021348  0.17856978]]
[[ 0.0300385   0.50738386 -0.04745137  0.39600455  0.04421249  0.70453695
  -0.00189001  0.98119386]]
2.6140288375305616
[2.428652595805809, 6.574126342742551, -1.4481197947844249, 10.163704774337253, 5.590376141929499, 20.449878048433213, -2.898010603842211, 16.116678567618795]
[[0.29508104 0.09922553 0.05615797 0.09652976 0.05645192 0.03864014
  0.11350587 0.24440777]]
[[ 0.71664933  0.65232115 -0.08132346  0.98109997  0.31558749  0.79018612
  -0.32894121  3.93904152]]
6.9846209093575755
[3.208252537101867, 13.62137260457368, -1.828869512199238, 30.789411554110988, 12.788974024813148, 36.179830207220114, -8.200415002992969, 39.77603101749069]
[[0.04846611 0.04146863 0.26135986 0.07957083 0.0690588  0.10007327
  0.1935228  0.20647971]]


In [53]:
types = ['Monthly', 'Quarterly', 'Half-Yearly', 'Yearly']
best_wts_for_avgs = []
annual_returns_for_avgs = []
port_vars_for_avgs = []
for avg in avgs:
    min_port_var = 1.62
    max_port_var = 1.75
    port_vars = []
    returns = []
    rand_wts = []
    for i in range(0, 10000):
        random_weights = np.random.dirichlet(np.ones(8), size = 1).T
        random_weighted_sd = stds * random_weights
        portfolio_var = np.sqrt(np.sum(np.dot(random_weighted_sd.T, np.dot(corr, random_weighted_sd))))*100
        if (portfolio_var >= min_port_var) & (portfolio_var <= max_port_var):
            port_vars.append(portfolio_var)
            rand_wts.append(random_weights)
            total_return = np.sum(avg * random_weights.T)
            returns.append(total_return)
    max_index = returns.index(max(returns))
    max_return = returns[max_index]
    best_wts = rand_wts[max_index]
    min_var = port_vars[max_index]

    best_wts_for_avgs.append(best_wts)
    port_vars_for_avgs.append(min_var)
    annual_ret = np.sum(mean_daily_returns * best_wts)*252
    annual_returns_for_avgs.append(annual_ret)

max_index = annual_returns_for_avgs.index(max(annual_returns_for_avgs))
maximum_return = annual_returns_for_avgs[max_index]
bestest_wts = best_wts_for_avgs[max_index]
miniest_var = port_vars_for_avgs[max_index]
print(f"Best Performance: {types[max_index]}")
print(f"Best Return: {maximum_return*100}%, Minimum Portfolio Variance: {miniest_var}")
print(f"Best Weights: {bestest_wts}, Sum of weights: {np.sum(bestest_wts)}")

Best Performance: Half-Yearly
Best Return: 46.050734215270076%, Minimum Portfolio Variance: 1.7489081803356825
Best Weights: [[0.09795665]
 [0.1665286 ]
 [0.01663003]
 [0.14351808]
 [0.10522839]
 [0.34196911]
 [0.03696478]
 [0.09120436]], Sum of weights: 1.0


In [54]:
ordered_types = {}
for i in range(4):
    ordered_types[types[i]] = annual_returns_for_avgs[i]*100
print(ordered_types)

{'Monthly': 45.061735277565205, 'Quarterly': 43.97870707786487, 'Half-Yearly': 46.050734215270076, 'Yearly': 44.06409383671666}
